In [23]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow


from sklearn.model_selection import train_test_split, GridSearchCV


import sys
sys.path.append('../')
from scripts import featurizer_rn


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df = pd.read_csv("../../data/dataset-procesado.csv", low_memory=False)

In [25]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.503081,0.122690,0.060852,0.818435,0.005692,0.150860,0.002214,0.150801,0.126017,0.275044,...,0.008680,0.004475,0.001958,0.009255,0.005948,dbc39818725bb5a6693839b2cc0aa6416619e257937f3b...,2017-07-31,CO,U,0
1,0.465945,0.002313,0.164618,0.002365,1.506865,0.477885,0.495657,0.047830,0.380410,0.111988,...,0.004941,1.007453,0.005539,0.007232,0.092750,f47daf0795add3305b3b61d7246eadbe470f7d23f720f7...,2017-11-28,CO,O,1
2,0.967004,0.147385,0.025202,1.006426,0.007549,0.112134,0.005367,0.071737,0.006661,0.034550,...,0.008593,0.006886,0.002667,0.009003,0.008484,cc6156327f2bb0f6d7026c8f4bd67925d07e90aa30c3f2...,2018-01-01,CL,O,0
3,0.606902,0.178559,0.200311,0.027147,0.009911,0.128841,0.003677,0.104828,0.257234,0.417678,...,0.006294,0.007027,0.006109,0.001858,0.001150,9a563afbb91edf3b91add14584e2cbead02cadecefa6a9...,2017-03-16,CO,O,1
4,0.499094,0.478672,0.040157,0.818853,0.008475,0.147698,0.003302,0.124946,0.255405,0.399290,...,0.006820,0.000544,0.004316,0.002098,0.008200,f83df258e87389552401e86124b840eb743986f025c230...,2017-09-01,CR,O,0


In [26]:
df.shape

(14739, 123)

In [27]:
X, Y = featurizer_rn.featurizer(df)

In [28]:
X.shape

(14739, 148)

In [29]:
dataset = X.values

In [30]:
Y

0        0
1        1
2        0
3        1
4        0
        ..
14734    1
14735    0
14736    0
14737    0
14738    1
Name: target, Length: 14739, dtype: int64

In [31]:
X = dataset.astype(float)
y = Y

In [32]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [33]:
for train, test in kfold.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

In [34]:
X.shape[0]

14739

In [35]:
X_train.shape[0]/X.shape[0]

0.8000542777664699

In [36]:
X_test.shape[0]/X.shape[0]

0.19994572223353008

In [42]:
# baseline model
def create_baseline(input_dim, opt):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=input_dim, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
    return model

In [43]:
from keras.optimizers import SGD
opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model_baseline = create_baseline(X_train.shape[1],opt)

In [44]:
model_baseline.fit(X_train, y_train, epochs=1000, verbose=1, batch_size=100,validation_split=0.2)

Epoch 1/1000
95/95 [==============================] - 0s 2ms/step - loss: 0.5439 - binary_accuracy: 0.7319 - val_loss: 0.4777 - val_binary_accuracy: 0.7533
Epoch 2/1000
95/95 [==============================] - 0s 901us/step - loss: 0.4477 - binary_accuracy: 0.7805 - val_loss: 0.4498 - val_binary_accuracy: 0.7864
Epoch 3/1000
95/95 [==============================] - 0s 878us/step - loss: 0.4232 - binary_accuracy: 0.7939 - val_loss: 0.4739 - val_binary_accuracy: 0.7685
Epoch 4/1000
95/95 [==============================] - 0s 911us/step - loss: 0.4141 - binary_accuracy: 0.8001 - val_loss: 0.4056 - val_binary_accuracy: 0.8046
Epoch 5/1000
95/95 [==============================] - 0s 835us/step - loss: 0.4093 - binary_accuracy: 0.8032 - val_loss: 0.4257 - val_binary_accuracy: 0.7948
Epoch 6/1000
95/95 [==============================] - 0s 930us/step - loss: 0.3984 - binary_accuracy: 0.8061 - val_loss: 0.4356 - val_binary_accuracy: 0.7940
Epoch 7/1000
95/95 [==============================] - 

95/95 [==============================] - 0s 922us/step - loss: 0.3701 - binary_accuracy: 0.8253 - val_loss: 0.3907 - val_binary_accuracy: 0.8181
Epoch 53/1000
95/95 [==============================] - 0s 948us/step - loss: 0.3672 - binary_accuracy: 0.8305 - val_loss: 0.4118 - val_binary_accuracy: 0.8152
Epoch 54/1000
95/95 [==============================] - 0s 976us/step - loss: 0.3693 - binary_accuracy: 0.8236 - val_loss: 0.3974 - val_binary_accuracy: 0.8181
Epoch 55/1000
95/95 [==============================] - 0s 932us/step - loss: 0.3681 - binary_accuracy: 0.8265 - val_loss: 0.3943 - val_binary_accuracy: 0.8190
Epoch 56/1000
95/95 [==============================] - 0s 915us/step - loss: 0.3677 - binary_accuracy: 0.8309 - val_loss: 0.3871 - val_binary_accuracy: 0.8262
Epoch 57/1000
95/95 [==============================] - 0s 885us/step - loss: 0.3689 - binary_accuracy: 0.8256 - val_loss: 0.4302 - val_binary_accuracy: 0.7940
Epoch 58/1000
95/95 [==============================] - 0s 93

95/95 [==============================] - 0s 971us/step - loss: 0.3654 - binary_accuracy: 0.8300 - val_loss: 0.3856 - val_binary_accuracy: 0.8262
Epoch 104/1000
95/95 [==============================] - 0s 920us/step - loss: 0.3627 - binary_accuracy: 0.8305 - val_loss: 0.3958 - val_binary_accuracy: 0.8211
Epoch 105/1000
95/95 [==============================] - 0s 921us/step - loss: 0.3624 - binary_accuracy: 0.8302 - val_loss: 0.3857 - val_binary_accuracy: 0.8270
Epoch 106/1000
95/95 [==============================] - 0s 994us/step - loss: 0.3628 - binary_accuracy: 0.8291 - val_loss: 0.3983 - val_binary_accuracy: 0.8105
Epoch 107/1000
95/95 [==============================] - 0s 945us/step - loss: 0.3623 - binary_accuracy: 0.8291 - val_loss: 0.3887 - val_binary_accuracy: 0.8266
Epoch 108/1000
95/95 [==============================] - 0s 945us/step - loss: 0.3633 - binary_accuracy: 0.8290 - val_loss: 0.3857 - val_binary_accuracy: 0.8220
Epoch 109/1000
95/95 [==============================] -

95/95 [==============================] - 0s 913us/step - loss: 0.3595 - binary_accuracy: 0.8326 - val_loss: 0.3827 - val_binary_accuracy: 0.8266
Epoch 155/1000
95/95 [==============================] - 0s 885us/step - loss: 0.3595 - binary_accuracy: 0.8317 - val_loss: 0.3992 - val_binary_accuracy: 0.8148
Epoch 156/1000
95/95 [==============================] - 0s 888us/step - loss: 0.3609 - binary_accuracy: 0.8296 - val_loss: 0.4011 - val_binary_accuracy: 0.8131
Epoch 157/1000
95/95 [==============================] - 0s 874us/step - loss: 0.3635 - binary_accuracy: 0.8308 - val_loss: 0.3865 - val_binary_accuracy: 0.8249
Epoch 158/1000
95/95 [==============================] - 0s 916us/step - loss: 0.3582 - binary_accuracy: 0.8311 - val_loss: 0.3854 - val_binary_accuracy: 0.8241
Epoch 159/1000
95/95 [==============================] - 0s 913us/step - loss: 0.3587 - binary_accuracy: 0.8337 - val_loss: 0.3951 - val_binary_accuracy: 0.8224
Epoch 160/1000
95/95 [==============================] -

95/95 [==============================] - 0s 897us/step - loss: 0.3557 - binary_accuracy: 0.8315 - val_loss: 0.3833 - val_binary_accuracy: 0.8292
Epoch 206/1000
95/95 [==============================] - 0s 871us/step - loss: 0.3564 - binary_accuracy: 0.8336 - val_loss: 0.4274 - val_binary_accuracy: 0.8131
Epoch 207/1000
95/95 [==============================] - 0s 960us/step - loss: 0.3563 - binary_accuracy: 0.8322 - val_loss: 0.3853 - val_binary_accuracy: 0.8321
Epoch 208/1000
95/95 [==============================] - 0s 911us/step - loss: 0.3551 - binary_accuracy: 0.8332 - val_loss: 0.3900 - val_binary_accuracy: 0.8232
Epoch 209/1000
95/95 [==============================] - 0s 890us/step - loss: 0.3543 - binary_accuracy: 0.8328 - val_loss: 0.4644 - val_binary_accuracy: 0.7719
Epoch 210/1000
95/95 [==============================] - 0s 982us/step - loss: 0.3561 - binary_accuracy: 0.8341 - val_loss: 0.3839 - val_binary_accuracy: 0.8309
Epoch 211/1000
95/95 [==============================] -

...

95/95 [==============================] - 0s 974us/step - loss: 0.3384 - binary_accuracy: 0.8366 - val_loss: 0.3973 - val_binary_accuracy: 0.8097
Epoch 767/1000
95/95 [==============================] - 0s 958us/step - loss: 0.3364 - binary_accuracy: 0.8405 - val_loss: 0.4208 - val_binary_accuracy: 0.8156
Epoch 768/1000
95/95 [==============================] - 0s 919us/step - loss: 0.3378 - binary_accuracy: 0.8389 - val_loss: 0.3957 - val_binary_accuracy: 0.8211
Epoch 769/1000
95/95 [==============================] - 0s 949us/step - loss: 0.3366 - binary_accuracy: 0.8409 - val_loss: 0.3979 - val_binary_accuracy: 0.8169
Epoch 770/1000
95/95 [==============================] - 0s 922us/step - loss: 0.3358 - binary_accuracy: 0.8381 - val_loss: 0.3972 - val_binary_accuracy: 0.8203
Epoch 771/1000
95/95 [==============================] - 0s 888us/step - loss: 0.3369 - binary_accuracy: 0.8385 - val_loss: 0.3970 - val_binary_accuracy: 0.8135
Epoch 772/1000
95/95 [==============================] -

95/95 [==============================] - 0s 941us/step - loss: 0.3355 - binary_accuracy: 0.8407 - val_loss: 0.3995 - val_binary_accuracy: 0.8186
Epoch 818/1000
95/95 [==============================] - 0s 938us/step - loss: 0.3348 - binary_accuracy: 0.8400 - val_loss: 0.4038 - val_binary_accuracy: 0.8148
Epoch 819/1000
95/95 [==============================] - 0s 893us/step - loss: 0.3377 - binary_accuracy: 0.8407 - val_loss: 0.3981 - val_binary_accuracy: 0.8211
Epoch 820/1000
95/95 [==============================] - 0s 895us/step - loss: 0.3363 - binary_accuracy: 0.8417 - val_loss: 0.4170 - val_binary_accuracy: 0.8139
Epoch 821/1000
95/95 [==============================] - 0s 929us/step - loss: 0.3360 - binary_accuracy: 0.8408 - val_loss: 0.3970 - val_binary_accuracy: 0.8198
Epoch 822/1000
95/95 [==============================] - 0s 944us/step - loss: 0.3374 - binary_accuracy: 0.8400 - val_loss: 0.4233 - val_binary_accuracy: 0.8131
Epoch 823/1000
95/95 [==============================] -

95/95 [==============================] - 0s 948us/step - loss: 0.3366 - binary_accuracy: 0.8385 - val_loss: 0.4086 - val_binary_accuracy: 0.8160
Epoch 869/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3344 - binary_accuracy: 0.8416 - val_loss: 0.4029 - val_binary_accuracy: 0.8173
Epoch 870/1000
95/95 [==============================] - 0s 980us/step - loss: 0.3334 - binary_accuracy: 0.8435 - val_loss: 0.4074 - val_binary_accuracy: 0.8114
Epoch 871/1000
95/95 [==============================] - 0s 923us/step - loss: 0.3381 - binary_accuracy: 0.8389 - val_loss: 0.3989 - val_binary_accuracy: 0.8203
Epoch 872/1000
95/95 [==============================] - 0s 941us/step - loss: 0.3333 - binary_accuracy: 0.8428 - val_loss: 0.4026 - val_binary_accuracy: 0.8156
Epoch 873/1000
95/95 [==============================] - 0s 931us/step - loss: 0.3328 - binary_accuracy: 0.8399 - val_loss: 0.3948 - val_binary_accuracy: 0.8181
Epoch 874/1000
95/95 [==============================] - 0

95/95 [==============================] - 0s 936us/step - loss: 0.3350 - binary_accuracy: 0.8383 - val_loss: 0.4133 - val_binary_accuracy: 0.8186
Epoch 920/1000
95/95 [==============================] - 0s 932us/step - loss: 0.3336 - binary_accuracy: 0.8396 - val_loss: 0.4050 - val_binary_accuracy: 0.8181
Epoch 921/1000
95/95 [==============================] - 0s 973us/step - loss: 0.3353 - binary_accuracy: 0.8417 - val_loss: 0.4039 - val_binary_accuracy: 0.8198
Epoch 922/1000
95/95 [==============================] - 0s 930us/step - loss: 0.3318 - binary_accuracy: 0.8426 - val_loss: 0.4221 - val_binary_accuracy: 0.7982
Epoch 923/1000
95/95 [==============================] - 0s 952us/step - loss: 0.3348 - binary_accuracy: 0.8407 - val_loss: 0.3984 - val_binary_accuracy: 0.8156
Epoch 924/1000
95/95 [==============================] - 0s 941us/step - loss: 0.3348 - binary_accuracy: 0.8429 - val_loss: 0.4098 - val_binary_accuracy: 0.8097
Epoch 925/1000
95/95 [==============================] -

95/95 [==============================] - 0s 1ms/step - loss: 0.3327 - binary_accuracy: 0.8393 - val_loss: 0.4039 - val_binary_accuracy: 0.8169
Epoch 971/1000
95/95 [==============================] - 0s 934us/step - loss: 0.3341 - binary_accuracy: 0.8431 - val_loss: 0.4027 - val_binary_accuracy: 0.8190
Epoch 972/1000
95/95 [==============================] - 0s 922us/step - loss: 0.3341 - binary_accuracy: 0.8437 - val_loss: 0.3993 - val_binary_accuracy: 0.8152
Epoch 973/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3301 - binary_accuracy: 0.8433 - val_loss: 0.3988 - val_binary_accuracy: 0.8173
Epoch 974/1000
95/95 [==============================] - 0s 934us/step - loss: 0.3336 - binary_accuracy: 0.8401 - val_loss: 0.4065 - val_binary_accuracy: 0.8135
Epoch 975/1000
95/95 [==============================] - 0s 984us/step - loss: 0.3325 - binary_accuracy: 0.8426 - val_loss: 0.4096 - val_binary_accuracy: 0.8173
Epoch 976/1000
95/95 [==============================] - 0s 

In [45]:
scores_train = model_baseline.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_train[1]*100))

369/369 [==============================] - 0s 526us/step - loss: 0.3413 - binary_accuracy: 0.8396

binary_accuracy: 83.96%


In [46]:
scores_test = model_baseline.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_test[1]*100))

93/93 [==============================] - 0s 502us/step - loss: 0.3816 - binary_accuracy: 0.8252

binary_accuracy: 82.52%
